# 04 - Estimation globale du paramètre xi

Dans ce notebook, nous allons construire un estimateur global pour le paramètre xi des GEV aux stations. Nous allons pour cela devoir construire une fonction à $2N+1$ paramètres, où $N$ est le nombre de stations : en effet, il faut estimer en même temps les paramètres $\mu_{i}$, $\phi_{i}$ et enfin $\xi$.

D'après notre précédente étude sur les corrélations des stations d'une même cellule, nous en sommes venus à la conclusion qu'il fallait mieux les concaténer, pour une même cellule. C'est donc ce que nous allons faire.

In [1]:
using CSV, DataFrames, StatsBase, Random

using Distributions, Extremes, Optim

using IDF

import Plots

In [77]:
PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]#provinces considerees
#PROVINCES = ["NB"]

DURATION = "24 h"

"24 h"

In [78]:
station_list = load_data("station_list")

filter!(row -> row.Province ∈ PROVINCES , station_list)#on ne selectionne que les stations qui nous interessent

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation
,String,String,String,Float64,Float64,Int64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91
2,BELLEDUNE,NB,8100514,47.9,-65.83,7
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154
7,FREDERICTON A,NB,8101500,45.87,-66.53,20
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70


In [4]:
first(load_station("8100512"), 10)#simple test

,Year,Duration,Pcp
,Int64,Cat…,Float64
1,1959,5 min,9.7
2,1960,5 min,5.3
3,1961,5 min,7.4
4,1962,5 min,5.6
5,1963,5 min,3.6
6,1964,5 min,3.6
7,1965,5 min,11.7
8,1966,5 min,3.0
9,1967,5 min,7.9


Il s'agit maintenant de calculer la log-vraisemblance à estimer, qui devra donc moduler $2N+1$ paramètres. En notant respectivement $\mu_{i}$ et $\phi_{i}$ les paramètres de moyenne et de variance, nous avons donc, avec $(y_{ij})_{1\leq j \leq n_{i}}$ les données de la $i$-ème station :

$$ l(y, \theta) = \sum_{i=1}^{N}\sum_{j=1}^{n_{i}} \ln(GEV(y_{ij}|\mu_{i}, \phi_{i}, \xi)) = \sum_{i=1}^{N} l(y_{i},(\mu_{i}, \phi_{i}, \xi)) $$

Avant de tester brutalement avec toutes les stations d'un coup, on pourrait tester par cellules ce que cela donne. Commencons par les extraire :

In [6]:
same_cell = load_data("same_cell")

first(same_cell, 10)

,Name,Province,ID,Lat,Lon,Elevation,GridCell
,String,String,String,Float64,Float64,Int64,Int64
1,ROYAL ROAD,NB,8104480,46.05,-66.72,115,33632
2,ROYAL ROAD WEST,NB,8104482,46.08,-66.73,160,33632
3,PORT WELLER (AUT),ON,6136699,43.25,-79.22,79,20197
4,ST CATHARINES A,ON,6137287,43.2,-79.17,97,20197
5,CAMBRIDGE GALT MOE,ON,6141095,43.33,-80.32,268,18645
6,PRESTON WPCP,ON,6146714,43.38,-80.35,272,18645
7,WATERLOO WELLINGTON A,ON,6149387,43.45,-80.38,317,18645
8,MAPLE,ON,6154950,43.87,-79.48,244,19620
9,TORONTO YORK MILLS,ON,615HHDF,43.75,-79.38,153,19620


In [7]:
cells = unique(same_cell[:, :GridCell])

14-element Vector{Int64}:
 33632
 20197
 18645
 19620
 19619
 19813
 19814
 19621
 28003
 28002
 25464
 28572
 26047
 27434

In [61]:
function xi_gridcell(GridCell::Int64)
    stations = filter(row -> row[:GridCell] == GridCell, same_cell)
    N = nrow(stations)
    
    # initialisation des parametres pour l'optimisation
    μ = []
    ϕ = []
    
    for i in 1:N
        y = pcp(stations[i, :ID], DURATION)
        append!(μ, mean(y))
        append!(ϕ, log(std(y)))
    end
    
    ξ = 0.0
    
    p₀ = Float64.([μ;ϕ;ξ])

    
    # definition de la fonction objectif
    # elle correspond a la log-vraisemblance globale définie plus haut
    function fobj(p::Vector{Float64})
        μ = p[1:N]
        ϕ = p[N+1:(2*N)]
        ξ = p[2*N+1]

        s=0
        for i in 1:N
            id = stations[i, :ID]
            s -= logL(pcp(id, DURATION), μ[i], exp(ϕ[i]), ξ) # on met l'opposé de la log-vraisemblance
        end

        return s
    end
        
    p = p₀
    try
        res = optimize(fobj, p₀)

        if Optim.converged(res)
            p = Optim.minimizer(res)
        else
            @warn "The algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
            p = p₀
        end
    catch
        println("Error with the cell number")
    end
    
    return p
end

xi_gridcell (generic function with 1 method)

In [63]:
for i in 1:(length(cells))
    println(xi_gridcell(cells[i]))
end

[48.829565157096845, 50.287583089785045, 2.5091186553695817, 2.413283679469214, 0.14609845653623607]
[44.35098603523281, 43.7493843726084, 2.346316679951111, 2.4525850213671614, -0.03760520748727994]
[48.42832230162551, 50.04331906154565, 45.39216305127042, 2.4391812119332683, 2.484676120016873, 2.5200091369007502, 0.07712823549998016]
[40.58052681781729, 32.19308030421434, 39.8425736089628, 2.1694631866499505, 2.635758148802551, 2.3979862770620586, 0.16763289234068687]
[41.105649679567456, 37.78852017526413, 2.320768210866288, 2.236615717917796, 0.1772713334494634]
[39.72641405358362, 37.64235199223889, 37.24331627803654, 2.3906879856630727, 2.1196232423602885, 2.21582854515578, 0.09377329096171523]
[39.48375259216612, 38.28886125049151, 2.0677309874160597, 1.9803808428574925, 0.07941146578639448]
[39.72237666243866, 40.00309237264619, 2.534799155167997, 2.3504691053030142, 0.07751214153358901]
[51.20282216510865, 53.29295114376827, 55.29778346055684, 55.9100393106775, 1.9241841707817

Maintenant que cette opération fonctionne bien avec les cellules contenant plusieurs stations, effectuons le calcul pour l'ensemble des stations.

D'après les recherches précédentes, nous allons donc considérer pour les cellules où il y a plusieurs stations, un agglomération sans plus d'affinement. Dans la suite du notebook, nous testerons à nouveau ce calcul avec une précision des cas multi-stations.

In [64]:
# rappel : nombre de stations total
nrow(station_list)

336

In [83]:
function xi_global(stations_df::DataFrame)
    N = nrow(stations_df)

    # initialisation des parametres
    μ = []
    ϕ = []

    for i in 1:N
        y = pcp(stations_df[i, :ID], DURATION)
        append!(μ, mean(y))
        append!(ϕ, log(std(y)))
    end

    ξ = 0.0

    p₀ = Float64.([μ;ϕ;ξ])


    # definition de la fonction objectif
    # elle correspond a la log-vraisemblance globale définie plus haut
    function fobj(p::Vector{Float64})
        μ = p[1:N]
        ϕ = p[N+1:(2*N)]
        ξ = p[2*N+1]

        s=0
        for i in 1:N
            id = stations_df[i, :ID]
            s -= logL(pcp(id, DURATION), μ[i], exp(ϕ[i]), ξ) # on met l'opposé de la log-vraisemblance
        end

        return s
    end

    p = p₀
    try
        res = optimize(fobj, p₀)

        if Optim.converged(res)
            p = Optim.minimizer(res)
        else
            @warn "The algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
            p = p₀
        end
    catch
        println("Error with the optimization")
    end

    return p
end

xi_global (generic function with 2 methods)

In [84]:
xi_global(station_list)

┌ Warning: The algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Main In[83]:42


673-element Vector{Float64}:
 65.07000000000001
 46.07222222222222
 60.42352941176471
 56.8235294117647
 59.770270270270274
 57.125
 60.690909090909095
 64.14230769230768
 62.468656716417904
 57.87692307692308
 58.43333333333334
 74.20500000000001
 79.23414634146341
  ⋮
  2.4971582908595926
  2.2505155731234727
  2.611911582985783
  2.47676473701447
  2.5557560236037307
  2.333751031523238
  2.3126877727380033
  2.509957210677259
  2.3748898571119708
  1.9682830066661798
  2.6818143009863102
  0.0

Comme l'optimisation semble ne pas converger du fait du trop grand nombre de stations, essayons par province séparément, pour des provinces ayant peu de stations :

In [80]:
combine(groupby(station_list, :Province), nrow)

,Province,nrow
,String,Int64
1,NB,15
2,NL,20
3,NS,16
4,ON,140
5,PE,3
6,QC,142


Testons donc avec l'île du Prince-Édouard, la province ayant le moins de stations (donc la plus à même de converger vers un résultat satisfaisant)

In [85]:
df_PE = filter(row -> row.Province == "PE" , station_list)
xi_global(df_PE)

7-element Vector{Float64}:
 54.72352334247815
 42.86725323886879
 51.84510845287662
  2.8808396405746035
  2.4425503151658687
  2.640542459739034
 -0.06798184896302174

Testons donc pour le nouveau-Brunswick :

In [86]:
df_NB = filter(row -> row.Province == "NB" , station_list)
xi_global(df_NB)

┌ Warning: The algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Main In[83]:42


31-element Vector{Float64}:
 65.07000000000001
 46.07222222222222
 60.42352941176471
 56.8235294117647
 59.770270270270274
 57.125
 60.690909090909095
 64.14230769230768
 62.468656716417904
 57.87692307692308
 58.43333333333334
 74.20500000000001
 79.23414634146341
  ⋮
  3.1589837390035154
  2.9246338982001756
  3.4057757500185
  2.981433350587477
  3.1636504360702293
  2.945388003370246
  2.780429385751191
  3.300971393315762
  3.2552740933625315
  2.6292454369441405
  2.865249192860548
  0.0

On observe que cela ne converge à nouveau pas. Testons pour la Nouvelle-Écosse : 

In [87]:
df_NS = filter(row -> row.Province == "NS" , station_list)
xi_global(df_NS)

┌ Warning: The algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values.
└ @ Main In[83]:42


33-element Vector{Float64}:
 57.74000000000001
 71.13846153846154
 60.76666666666667
 75.80434782608695
 77.25833333333333
 88.19333333333334
 61.38809523809524
 67.5
 72.62307692307694
 78.1
 71.15254237288136
 65.54375
 71.77169811320753
  ⋮
  3.3027103008700758
  2.9415927082583866
  2.938005024152091
  3.0732652255850446
  3.373245232356896
  3.1935390216679025
  2.6615861821636018
  3.186242740430084
  2.848603289893488
  3.4793211906805364
  2.829443919657416
  0.0